In [35]:
# General
import string
import numpy as np
import pandas as pd
import warnings

# Ignore warnings in libraries
warnings.filterwarnings("ignore")

# Gensim
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, KeyedVectors
from gensim.models.wrappers import LdaMallet
from gensim.test.utils import datapath
# Lemmatization
import spacy

# Plotting tools
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [36]:
data = pd.read_csv("~/Desktop/Svevo/carteggio.svevo3.csv", sep=';', parse_dates=['date'])
nlp_it = spacy.load("it", disable=['parser', 'ner'])
stop_words = set(['schmitz', 'signore', 'signora', 'ettore', 'lettera', 'parola', 'fare', 'cosa'])

In [37]:
def preprocess_text(text, nlp, stop_words = [], pos = ['PROPN', 'NOUN', 'VERB', 'ADJ']):
    """
    Performs preprocessing on a text through spaCy, langauge based on nlp parameter.
    Filter default stopword plus those in the list stop_words,
    accepts only word with POS tag in the pos list,
    filters out non-alpha words, performs lemmatization.
    Returns a list of lemmatized, filtered tokens for the text.
    """
    doc = nlp(text.lower())
    tokens = []
    for token in doc:
        if token.lemma_ not in stop_words and token.pos_ in pos and not token.is_stop and token.is_alpha:
            tokens.append(token.lemma_)
    return tokens

def create_lda_model(dictionary, corpus, num_topics, num_words = 20, passes = 20):
    """
    Creates an LDA model using gensim.
    Dictionary and corpus should be obtained with corpora.
    Number of topics and number of passes should be tweaked depending on context.
    """
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes, random_state=42)
    for topic,words in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=num_words):
        print(str(topic)+ ": " + words + "\n")
    return lda_model

def get_lda_keywords(model, num_words = 20):
    """
    Extract the firt num_words keywords from all topics of a gensim LDA model.
    """
    num_topics = len(model.print_topics())
    return [[tup2[0] for tup2 in tup[1]]for tup in model.show_topics(formatted=False, num_topics=num_topics, num_words=num_words)]
    
def create_paper_points_data(lda_model, corpus, num_topics):
    """
    Creates a dataframe with topic scores for each text in corpus using a gensim LDA model.
    """
    x = [[] for i in range(num_topics)]
    for text in corpus:
        l_tup = lda_model[text]
        for val in range(num_topics):
            added = False
            for tup in l_tup:
                if tup[0] == val:
                    x[val].append(tup[1])
                    added = True
            if added == False:
                x[val].append(0)
    df = pd.DataFrame(x)
    df = df.transpose()
    df.columns = ['Topic' + str(n) for n in range(num_topics)]
    return df

def compute_silhouette(lda_model, corpus, num_topics):
    """
    Computes silhouette index for an LDA model based on topics it classified.
    """
    points = create_paper_points_data(lda_model, corpus, num_topics).values
    lda_labels = points.argmax(axis=1) 
    return silhouette_score(points, lda_labels)
    
def compute_perplexity(lda_model, corpus):
    """
    Computes perplexity score for an LDA model.
    """
    perplexity = lda_model.log_perplexity(corpus)
    return perplexity

def compute_coherence(lda_model, texts, corpus, dictionary, coherence = "c_v"):
    """
    Computes coherence score for an LDA model.
    """
    score = CoherenceModel(model=lda_model, texts=texts, corpus=corpus, dictionary=dictionary, coherence=coherence).get_coherence()
    return score


In [41]:
data_it = data[data.mainLanguage == "ITA"]
texts_it = [preprocess_text(text, nlp_it, stop_words=stop_words, pos = ['PROPN', 'NOUN']) for text in data_it["text"]]
d_it = corpora.Dictionary(texts_it)
d_it.filter_extremes(no_below=5, no_above=0.5)
c_it = [d_it.doc2bow(text) for text in texts_it]


In [79]:

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaModel(corpus, num_topics, id2word=dictionary, passes=200, random_state=42)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [80]:
model_list , coherence_values = compute_coherence_values(d_it, c_it , texts_it , 10 , 2 , 1)

In [83]:
vis = pyLDAvis.gensim.prepare(model_list[0] , c_it , d_it)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.145839  0.0       1        1  75.600565
0     -0.145839  0.0       2        1  24.399435, topic_info=           Term        Freq       Total Category  logprob  loglift
51      amicare  130.000000  130.000000  Default  30.0000  30.0000
74     senilità  120.000000  120.000000  Default  29.0000  29.0000
152  articolare  115.000000  115.000000  Default  28.0000  28.0000
48      trieste  398.000000  398.000000  Default  27.0000  27.0000
29       parigi  120.000000  120.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
174     operare   54.122790  136.500442   Topic2  -5.0190   0.4855
42     scrivere   40.994430   86.677139   Topic2  -5.2968   0.6619
89       milano   39.464162   98.303780   Topic2  -5.3349   0.4979
69      piacere   42.427248  171.581320   Topic2  -5.2625   0.0133
115      londra   38.488345  185.206619   Topic2  -5.3599  -0.1605

[127 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
853       1  0.998580  abbracciare
853       2  0.004688  abbracciare
325       1  0.965778        acqua
325       2  0.030498        acqua
968       1  0.993629       adolfo
...     ...       ...          ...
364       1  0.931897    viaggiare
364       2  0.070777    viaggiare
276       1  0.013832        villa
276       2  0.995895        villa
104       2  0.985575         zeno

[151 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [86]:
data_it.groupby([data_it.year]).agg({'count'})

,n,corpus,index,date,sender,senderLocation,recipient,recipientLocation,languages,mainLanguage,text
,count,count,count,count,count,count,count,count,count,count,count
year,,,,,,,,,,,
1885,1,1,1,1,1,1,1,1,1,1,1
1886,1,1,1,1,1,1,1,1,1,1,1
1895,9,9,9,9,9,9,9,9,9,9,9
1896,23,23,23,23,23,23,23,23,23,23,23
1897,10,10,10,10,10,10,10,10,10,10,10
1898,40,40,40,40,40,40,40,40,40,40,40
1899,28,28,28,28,28,28,28,28,28,28,28
1900,35,35,35,35,35,35,35,35,35,35,35
